# Figure of station profile model evaluation


In [ ]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import xarray as xr
import pickle
import glob
from datetime import datetime
import modules.mapping_functions as mf
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

#### Parameters:

In [ ]:
# domain dimensions:
imin, imax = 1479, 2179
jmin, jmax = 159, 799

# results to plot:
folder_ref_2015      = f'/data/brogalla/run_storage/Mn-reference-202204/ANHA12_ref-2015_20220422/'
folder_ref_2009      = f'/data/brogalla/run_storage/Mn-reference-202204/ANHA12_ref-2009_20220422/'
folder_cleanice_2009 = '/data/brogalla/run_storage/Mn-clean-ice-202204/ANHA12_clean-ice-2009_20220422/'
folder_cleanice_2015 = '/data/brogalla/run_storage/Mn-clean-ice-202204/ANHA12_clean-ice-2015_20220422/'
folder_spm_2009      = '/data/brogalla/run_storage/Mn-spm-202204/ANHA12_spm-2009_20220422/'
folder_spm_2015      = '/data/brogalla/run_storage/Mn-spm-202204/ANHA12_spm-2015_20220422/'
folder_bio_2009      = '/data/brogalla/run_storage/Mn-bio-202204/ANHA12_bio-2009_20220422/'
folder_bio_2015      = '/data/brogalla/run_storage/Mn-bio-202204/ANHA12_bio-2015_20220422/'

# colours:
c_continent    = '#ce9169'
c_glacier      = '#36ab92'
c_other        = '#a6a6a6'
land_color     = "#8b7765"
profile_colors = ['#e54141', '#33c7ff']
rivers         = '#bada55'
dirty          = '#64707b'
clean          = '#cee7fd'

#### Load files:

Model results

In [ ]:
model_ref_2015 = nc.Dataset(f'{folder_ref_2015}ANHA12_EXH006_2015_monthly.nc')
lon_model    = np.array(model_ref_2015.variables['nav_lon'])
lat_model    = np.array(model_ref_2015.variables['nav_lat'])
depths_model = np.array(model_ref_2015.variables['deptht'])

Meshes

In [ ]:
# ANHA12 grid
mesh       = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
mesh_lon   = np.array(mesh.variables['nav_lon'])
mesh_lat   = np.array(mesh.variables['nav_lat'])
mesh_bathy = np.array(mesh.variables['hdept'][0])
tmask    =  np.array(mesh.variables['tmask'])[0,:,imin:imax,jmin:jmax]
mlons    = np.array(mesh.variables['nav_lon'])
mlats    = np.array(mesh.variables['nav_lat'])
Z_masked = np.ma.masked_where((tmask > 0.1), tmask) 

Observations

In [ ]:
# Concentration of dMn in the Beaufort Sea in 2009 (data from Nari Sim's thesis: http://hdl.handle.net/2429/67889
IPY = pd.read_csv('/ocean/brogalla/GEOTRACES/data/Nari_Sim_dMn_data.csv')
IPY_names_full = IPY['Station name'].astype('string').values
IPY_depths     = IPY['Depth [m]'].astype('float').values
IPY_dMn_nmolkg = IPY['DMn [ nmol/kg]'].astype('float').values
IPY_lons_full  = -1*IPY['Longitude'].astype('float').values
IPY_lats_full  = IPY['Latitude'].astype('float').values
# Unique entries (maintaining order)
IPY_lons  = np.array(list(dict.fromkeys(IPY_lons_full)))
IPY_lats  = np.array(list(dict.fromkeys(IPY_lats_full)))
IPY_names = np.array(list(dict.fromkeys(IPY_names_full)))

# Convert nmol / kg to nmol / L
IPY_density             = IPY['Density'].astype('float').values - 1000 # kg / L
IPY_depths_cruise_sheet = IPY['Depth [m] cruise sheet'].astype('float').values
IPY_dMn_nmolL = IPY_dMn_nmolkg * (IPY_density+1000)/1000

In [ ]:
# Concentration of dMn in 2015 (data from Colombo et al., 2020)
Manuel = pd.read_csv('/ocean/brogalla/GEOTRACES/data/DMn_nmolL-1_BR-edited.csv')
Manuel_station_names = Manuel['Station'].astype('string')
Manuel_depths        = Manuel['Depth'].astype('float')
Manuel_dMn_nmolkg    = Manuel['nmol kg-1'].astype('float').values
Manuel_dMn_nmolL     = Manuel['nmol L-1'].astype('float').values
Manuel_salinity      = Manuel['Salinity'].astype('float').values

# Station information:
GEOTRACES_CTD = pd.read_csv('/ocean/brogalla/GEOTRACES/data/combined.csv')
CTD_stations  = GEOTRACES_CTD['Station'].astype('string')
CTD_temp      = GEOTRACES_CTD['Temperature [ITS-90, deg C]'].astype('float')
CTD_sal       = GEOTRACES_CTD['Salinity, Practical [PSU]'].astype('float')
CTD_depth     = GEOTRACES_CTD['Depth [salt water, m], lat = 49.00'].astype('float')

GEOTRACES = pd.read_csv('/ocean/brogalla/GEOTRACES/data/Pb-paper-data.csv')
stn_names = GEOTRACES['Station names'].dropna().astype('str')
lons      = GEOTRACES['station lon'].astype('float').dropna().values
lats      = GEOTRACES['Station lat'].astype('float').dropna().values

#### Calculations:

In [ ]:
def calculate_average(folder_ref_2009, folder_ref_2015):
    # Calculate average over data files during cruise period:
    # 2009 GEOTRACES: 29 August - 8 September
    files_2009      = np.sort(glob.glob(f'{folder_ref_2009}ANHA12_EXH006_5d_20090101_20091231*'))
    start_date_2009 = datetime.strptime('2009-08-29', '%Y-%m-%d')
    end_date_2009   = datetime.strptime('2009-09-08', '%Y-%m-%d')

    file_list_2009 = []
    for file in files_2009:
        if (datetime.strptime(file.split('/')[-1][42:50],'%Y%m%d') >= start_date_2009) & \
                (datetime.strptime(file.split('/')[-1][51:59],'%Y%m%d') < end_date_2009):
            file_list_2009.append(file)
    dmn_2009 = np.zeros((50,700,640))
    days = 0
    for file in file_list_2009:
        ds = xr.open_dataset(f"{file}")
        dmn_2009 = dmn_2009 + ds['dissolmn'].values[0,:,:,:]

    mn_dis_ref_2009 = dmn_2009 / len(file_list_2009)

    # 2015 GEOTRACES: 9 August - 15 September
    files_2015      = np.sort(glob.glob(f'{folder_ref_2015}ANHA12_EXH006_5d_20150101_20151231*'))
    start_date_2015 = datetime.strptime(f'2015-08-09', '%Y-%m-%d')
    end_date_2015   = datetime.strptime(f'2015-09-15', '%Y-%m-%d')

    file_list_2015 = []
    for file in files_2015:
        if (datetime.strptime(file.split('/')[-1][42:50],'%Y%m%d') >= start_date_2015) & \
                    (datetime.strptime(file.split('/')[-1][51:59],'%Y%m%d') < end_date_2015):
            file_list_2015.append(file)

    dmn_2015 = np.zeros((50,700,640))
    days = 0
    for file in file_list_2015:
        ds = xr.open_dataset(f"{file}")
        dmn_2015 = dmn_2015 + ds['dissolmn'].values[0,:,:,:]

    mn_dis_ref_2015 = dmn_2015 / len(file_list_2015)
    
    return mn_dis_ref_2009, mn_dis_ref_2015

In [ ]:
mn_ref_2009, mn_ref_2015           = calculate_average(folder_ref_2009, folder_ref_2015)
mn_cleanice_2009, mn_cleanice_2015 = calculate_average(folder_cleanice_2009, folder_cleanice_2015)
mn_spm_2009, mn_spm_2015           = calculate_average(folder_spm_2009, folder_spm_2015)
mn_bio_2009, mn_bio_2015           = calculate_average(folder_bio_2009, folder_bio_2015)

In [ ]:
# Connect these points with straight line segments:
transect_lons = [lons[stn_names=='CB4'], lons[stn_names=='CB3'],lons[stn_names=='CB2'], lons[stn_names=='CB1'], \
                 lons[stn_names=='CAA8'], lons[stn_names=='CAA6'], lons[stn_names=='CAA7'], lons[stn_names=='CAA4'],\
                 lons[stn_names=='CAA5'], lons[stn_names=='CAA1'], lons[stn_names=='CAA2'], lons[stn_names=='CAA3']]
transect_lats = [lats[stn_names=='CB4'], lats[stn_names=='CB3'],lats[stn_names=='CB2'], lats[stn_names=='CB1'], \
                 lats[stn_names=='CAA8'], lats[stn_names=='CAA6'], lats[stn_names=='CAA7'], lats[stn_names=='CAA4'],\
                 lats[stn_names=='CAA5'], lats[stn_names=='CAA1'], lats[stn_names=='CAA2'], lats[stn_names=='CAA3']]
transect_names = ['CB4','CB3', 'CB2', 'CB1', 'CAA8', 'CAA6', 'CAA7', 'CAA4', 'CAA5', 'CAA1', 'CAA2', 'CAA3']

In [ ]:
# Create vectors of many points along these segments:
transect_vlons = np.array([])
transect_vlats = np.array([])
for i in range(0,len(transect_lons)-1):
    diff_lons = transect_lons[i+1] - transect_lons[i]
    diff_lats = transect_lats[i+1] - transect_lats[i]
    vector_lons = transect_lons[i] + np.linspace(0,diff_lons,30)
    vector_lats = transect_lats[i] + np.linspace(0,diff_lats,30)
    transect_vlons = np.append(transect_vlons, vector_lons)
    transect_vlats = np.append(transect_vlats, vector_lats)

#### Define functions

In [ ]:
def station_profile(ax, name, manuel=False, patch=False, full=True, default_label=True):
    
    ax.invert_yaxis()
    ax.tick_params(labelsize=6, pad=0)
    ax.set_xlim([-2, 15])
    ax.xaxis.tick_top()    
    ax.xaxis.set_label_position('top') 
    ax.set_xticks([0,5,10,15])
       
    # observations: -------------------------------------------------------------------------------------------
    if manuel: # 2015 cruise
        dMn_obs   = Manuel_dMn_nmolL[np.array(Manuel_station_names==name).astype('bool')]
        depth_obs = Manuel_depths[np.array(Manuel_station_names==name).astype('bool')].values
    else:      # 2009 cruise
        dMn_obs   = IPY_dMn_nmolL[np.array(IPY_names_full==name).astype('bool')]
        depth_obs = IPY_depths_cruise_sheet[np.array(IPY_names_full==name).astype('bool')]

    ax.plot(dMn_obs, depth_obs, '.-', 
            zorder=4, markeredgewidth=0.5, markeredgecolor='k', color=profile_colors[0], markersize=7, linewidth=1.8)

    plot_depth_lowerrange = depth_obs[-1] # deepest observation    
    plot_depth_upperrange = depth_obs[0]  # shallowest observation
    
    # model: --------------------------------------------------------------------------------------------------
    if manuel:  # 2015 model results
        lon_station = lons[stn_names==name]
        lat_station = lats[stn_names==name]

        model_stni, model_stnj = mf.find_indeces_vector(lon_station, lat_station, lon_model, lat_model)
        model_stni = int(model_stni); model_stnj = int(model_stnj);

        mn_ref = mn_ref_2015[:, model_stni, model_stnj]*10**9 
    else:       # 2009 model results
        lon_station = IPY_lons[IPY_names==name]
        lat_station = IPY_lats[IPY_names==name]

        model_stni, model_stnj = mf.find_indeces_vector(lon_station, lat_station, lon_model, lat_model)
        model_stni = int(model_stni); model_stnj = int(model_stnj);

        mn_ref = mn_ref_2009[:, model_stni, model_stnj]*10**9 

    mn_ref_masked = np.ma.masked_where((tmask[:,model_stni, model_stnj] < 0.1), mn_ref)
    
    ################### full water column #############################
    if full:
        # Run with sediment in sea ice:
        ax.plot(mn_ref_masked[depths_model > plot_depth_upperrange], \
                depths_model[depths_model > plot_depth_upperrange], '.-', \
                zorder=1, linewidth=1.8, color=profile_colors[1], markeredgecolor='k', markeredgewidth=0.5, markersize=7)
        
        ax.set_ylim([plot_depth_lowerrange*1.2, 0-(plot_depth_lowerrange*0.04)])
        
        if default_label:
            ax.text(5.5, plot_depth_lowerrange*0.99, name, fontsize=6)

        if patch:
            # Background patches behind profiles: ----------------------------------------------------------------------
            #top patch:
            x0_top     = -11
            y0_top     = -(plot_depth_lowerrange*0.3)
            width_top  = 15+12.4
            height_top = (plot_depth_lowerrange*0.3)
            box_top    = mpl.patches.Rectangle((x0_top,y0_top), width_top, height_top, facecolor='w', clip_on=False, \
                                              alpha=0.9, zorder=1)
            ax.add_patch(box_top)

            #left side patch:
            x0_side     = -11
            y0_side     = 0
            width_side  = 15+12.4
            height_side = plot_depth_lowerrange*1.24
            box_side    = mpl.patches.Rectangle((x0_side,y0_side), width_side, height_side, \
                                                facecolor='w', clip_on=False, alpha=0.9, zorder=1)
            ax.add_patch(box_side)
       
    ################### zoom in on the surface #############################
    else:
        # Run with sediment in sea ice:
        ax.plot(mn_ref_masked[depths_model > plot_depth_upperrange], \
                depths_model[depths_model > plot_depth_upperrange], '.-', \
                zorder=1, linewidth=1.8, color=profile_colors[1], markeredgecolor='k', markeredgewidth=0.5, markersize=7)
        
        ax.set_ylim([100, 0])
        ax.text(5.5, 96, name, fontsize=6)

        if patch:
        # Background patches behind profiles: ----------------------------------------------------------------------
            #top patch:
            x0_top     = -11;     y0_top     = -30;
            width_top  = 15+12.4; height_top = 30;
            box_top    = mpl.patches.Rectangle((x0_top,y0_top), width_top, height_top, facecolor='w', clip_on=False, \
                                               alpha=0.9, zorder=1)
            ax.add_patch(box_top)
            
            #left side patch:
            x0_side     = -11;     y0_side     = 0;
            width_side  = 15+12.4; height_side = 105;
            box_side    = mpl.patches.Rectangle((x0_side,y0_side), width_side, height_side, \
                                                facecolor='w', clip_on=False, alpha=0.9, zorder=1)
            ax.add_patch(box_side)
    return

In [ ]:
def full_figure(ln_full=True, savefig=False, title=''):
    
    fig, ax, proj1, proj2 = pickle.load(open('/ocean/brogalla/GEOTRACES/pickles/mn-eval-combined.pickle','rb'))
    #--------------------------------------------------------------------------------------------------------------------------
    ### Left plot ###

    # profile plots:
    ax1 = plt.axes([0.155, 0.15, 0.05, 0.17]) #L1
    ax2 = plt.axes([0.155, 0.39, 0.05, 0.17]) #L1.1 
    ax3 = plt.axes([0.155, 0.66, 0.05, 0.17]) #L1.5 
    ax4 = plt.axes([0.3, 0.66, 0.05, 0.17])  #L2
    ax5 = plt.axes([0.3, 0.39, 0.05, 0.17]) #L3
    ax6 = plt.axes([0.3, 0.15, 0.05, 0.17])  #S4
    axes_list = [ax1, ax2, ax3, ax4, ax5, ax6]

    fig1 = station_profile(ax1, 'L1',   full=ln_full)
    fig2 = station_profile(ax2, 'L1.1', full=ln_full)
    fig3 = station_profile(ax3, 'L1.5', full=ln_full)
    fig4 = station_profile(ax4, 'L2',   full=ln_full)
    fig5 = station_profile(ax5, 'L3',   full=ln_full)
    fig6 = station_profile(ax6, 'S4', patch=True, full=ln_full)

    x_GEOTRACES, y_GEOTRACES = proj1(lons, lats)
    x_IPY      , y_IPY       = proj1(IPY_lons, IPY_lats)
    ax[0].scatter(x_GEOTRACES, y_GEOTRACES, marker='*', s=60, zorder=7, edgecolor='k', linewidth=0.5, color='w', label='2015 GEOTRACES')
    ax[0].scatter(x_IPY      , y_IPY      , marker='*', s=60, zorder=7, edgecolor='k', linewidth=0.5, color='#848484', label='2009 IPY GEOTRACES')
    ax[0].legend(loc=(-0.03, 1.02), fontsize=7, frameon=False)

    # create lines connecting the profiles:
    coordsA = "data"
    coordsB = "data"
    for i in range(len(IPY_names)):
        xy1 = (x_IPY[i], y_IPY[i])
        if ln_full:
            xy2 = (10, 200)
        else:
            xy2 = (10, 50)
        axB = axes_list[i]
        con = ConnectionPatch(xyA=xy1, xyB=xy2, axesA=ax[0], axesB=axB, \
                              coordsA=coordsA, coordsB=coordsB, linestyle="dotted", linewidth=1.0, zorder=3)
        ax[0].add_artist(con)

#     ax[0].text(1.1e5, 0.75e6, 'Dissolved Mn [nM]', fontsize=8)
#     ax[0].text(0.1e5, 0.50e6, 'Depth [m]', fontsize=8, rotation=90)

    #--------------------------------------------------------------------------------------------------------------------------
    ### Right plot ###
    x_GEOTRACES, y_GEOTRACES = proj2(lons, lats)
    x_IPY      , y_IPY       = proj2(IPY_lons, IPY_lats)
    ax[1].scatter(x_GEOTRACES, y_GEOTRACES, marker='*', s=60, zorder=7, edgecolor='k', linewidth=0.5, color='w')
    ax[1].scatter(x_IPY      , y_IPY      , marker='*', s=60, zorder=7, edgecolor='k', linewidth=0.5, color='#848484')

    # profile plots:
    ax1 = plt.axes([0.84, 0.65, 0.05, 0.17]) # CAA1
    ax2 = plt.axes([0.84, 0.38, 0.05, 0.17]) 
    ax3 = plt.axes([0.84, 0.14, 0.05, 0.17]) 
    ax4 = plt.axes([0.71, 0.14, 0.05, 0.17])
    ax5 = plt.axes([0.765, 0.65, 0.05, 0.17])
    ax6 = plt.axes([0.62, 0.14, 0.05, 0.17]) 
    ax7 = plt.axes([0.54, 0.14, 0.05, 0.17])
    ax8 = plt.axes([0.46, 0.14, 0.05, 0.17])
    ax9 = plt.axes([0.69, 0.65, 0.05, 0.17])
    ax10 = plt.axes([0.61, 0.65, 0.05, 0.17]) #CB1
    ax11 = plt.axes([0.46, 0.36, 0.05, 0.17])
    ax12 = plt.axes([0.54, 0.76, 0.05, 0.17])
    ax13 = plt.axes([0.46, 0.76, 0.05, 0.17])

    axes_list=[ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12, ax13]

    fig1 = station_profile(ax1, 'CAA1', manuel=True, patch=True, full=ln_full)
    fig2 = station_profile(ax2, 'CAA2', manuel=True, patch=True, full=ln_full, default_label=False)
    fig3 = station_profile(ax3, 'CAA3', manuel=True, patch=True, full=ln_full, default_label=False)
    fig4 = station_profile(ax4, 'CAA4', manuel=True, patch=True, full=ln_full, default_label=False)
    fig5 = station_profile(ax5, 'CAA5', manuel=True, patch=True, full=ln_full, default_label=False)
    fig6 = station_profile(ax6, 'CAA6', manuel=True, patch=True, full=ln_full, default_label=False)
    fig7 = station_profile(ax7, 'CAA7', manuel=True, patch=True, full=ln_full, default_label=False)
    fig8 = station_profile(ax8, 'CAA8', manuel=True, patch=True, full=ln_full, default_label=False)
    fig9 = station_profile(ax9, 'CAA9', manuel=True, full=ln_full, default_label=False)
    fig10 = station_profile(ax10, 'CB1', manuel=True, full=ln_full)
    fig11 = station_profile(ax11, 'CB2', manuel=True, patch=True, full=ln_full)
    fig12 = station_profile(ax12, 'CB3', manuel=True, full=ln_full)
    fig13 = station_profile(ax13, 'CB4', manuel=True, full=ln_full)
    
    ax2.text(5.5, 400, 'CAA2', fontsize=6)
    ax3.text(5.5, 440, 'CAA3', fontsize=6)
    ax4.text(5.5, 120, 'CAA4', fontsize=6)
    ax5.text(6.1, 155, 'CAA5', fontsize=6)
    ax6.text(6.1, 90, 'CAA6', fontsize=6)
    ax7.text(5.5, 150, 'CAA7', fontsize=6)
    ax8.text(4.5, 350, 'CAA8', fontsize=6)
    ax9.text(5.6, 340, 'CAA9', fontsize=6)

    # create lines connecting the profiles:
    coordsA = "data"
    coordsB = "data"
    for i in range(len(stn_names[5:18])):
        xy1 = (x_GEOTRACES[i+5], y_GEOTRACES[i+5])
        xy2 = (10, 50)#300/2)
        axB = axes_list[i]
        con = ConnectionPatch(xyA=xy1, xyB=xy2, axesA=ax[1], axesB=axB, \
                              coordsA=coordsA, coordsB=coordsB, linestyle="dotted", linewidth=1.0, zorder=2)
        ax[1].add_artist(con)

    # Transect: 
    xt, yt = proj2(transect_vlons, transect_vlats)
    proj2.plot(xt, yt, c=profile_colors[0], linewidth=2, zorder=2)  

    # legend
    ax[1].plot(0,0,'-', c=profile_colors[0], linewidth=2.5, label='Observed dMn')
    ax[1].plot(0,0,'-', c=profile_colors[1], linewidth=2.5, label='Modelled dMn')
    ax[1].legend(loc=(-0.33, 1.01), fontsize=7, frameon=False, handlelength=0.7)

    #-----------------------------------------------------------------------------------------------------------------------------------
    ### Full figure ###
    fig.text(0.1, 0.95, 'a)', fontsize=9);
    fig.text(0.4, 0.95, 'b)', fontsize=9);
    
    fig.text(0.3, 1.00, title, fontsize=9);

    if savefig:
        fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/R1-evaluation-profiles.jpg', \
                    dpi=300, bbox_inches='tight')
        fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/R1-evaluation-profiles.svg', \
                    dpi=300, bbox_inches='tight', format='svg')
        
    return

#### Figure:

In [ ]:
full_figure(ln_full=True, title='', savefig=True)

# Scatter plot of observations

In [ ]:
def calc_scatter(name):    
    try:
        # 2009 data:
        a = not(Manuel_dMn_nmolL[np.array(Manuel_station_names==name).astype('bool')]) # returns true if name is not in list of station names
        print(f'data from IPY 2009 stn {name}')
        # observations
        dmn_stn   = IPY_dMn_nmolL[np.array(IPY_names_full==name).astype('bool')]
        depth_stn = IPY_depths_cruise_sheet[np.array(IPY_names_full==name).astype('bool')]
        lon_station = IPY_lons[IPY_names==name]
        lat_station = IPY_lats[IPY_names==name] 
        
        # model data:
        model_stni, model_stnj = mf.find_indeces_vector(lon_station, lat_station, lon_model, lat_model)
        model_stni = int(model_stni); model_stnj = int(model_stnj);
        mn_ref      = mn_ref_2009[:,model_stni,model_stnj]*10**9
        mn_cleanice = mn_cleanice_2009[:,model_stni,model_stnj]*10**9
        mn_spm      = mn_spm_2009[:,model_stni,model_stnj]*10**9
        mn_bio      = mn_bio_2009[:,model_stni,model_stnj]*10**9
    except:
        # 2015 data:
        print(f'data from GEOTRACES 2015 stn {name}')
        # observations
        dmn_stn   = Manuel_dMn_nmolL[np.array(Manuel_station_names==name).astype('bool')]
        depth_stn = Manuel_depths[np.array(Manuel_station_names==name).astype('bool')]
        lon_station = lons[stn_names==name]
        lat_station = lats[stn_names==name]        
        
        # model data:
        model_stni, model_stnj = mf.find_indeces_vector(lon_station, lat_station, lon_model, lat_model)
        model_stni = int(model_stni); model_stnj = int(model_stnj);
        mn_ref      = mn_ref_2015[:,model_stni,model_stnj]*10**9
        mn_cleanice = mn_cleanice_2015[:,model_stni,model_stnj]*10**9
        mn_spm      = mn_spm_2015[:,model_stni,model_stnj]*10**9
        mn_bio      = mn_bio_2015[:,model_stni,model_stnj]*10**9

    mn_ref_masked      = np.ma.masked_where((tmask[:,model_stni, model_stnj] < 0.1), mn_ref)
    mn_cleanice_masked = np.ma.masked_where((tmask[:,model_stni, model_stnj] < 0.1), mn_cleanice)
    mn_spm_masked      = np.ma.masked_where((tmask[:,model_stni, model_stnj] < 0.1), mn_spm)
    mn_bio_masked      = np.ma.masked_where((tmask[:,model_stni, model_stnj] < 0.1), mn_bio)
    
    # match data:
    depth_lower = 50
    depth_upper = 5
    dmn_refm = []; dmn_cleanm = []; dmn_spmm = []; dmn_biom = [];
    for depth_obs in depth_stn[(depth_stn < depth_lower) & (depth_stn > depth_upper)]:
        d1 = np.argmin(np.abs(depths_model-depth_obs))
        dmn_refm.append(mn_ref_masked[d1])
        dmn_cleanm.append(mn_cleanice_masked[d1])
        dmn_spmm.append(mn_spm_masked[d1])
        dmn_biom.append(mn_bio_masked[d1])
    
    dmn_obs = list(dmn_stn[(depth_stn < depth_lower) & (depth_stn > depth_upper)])
    
    return dmn_refm, dmn_cleanm, dmn_spmm, dmn_biom, dmn_obs

In [ ]:
stations_2015  = ['CAA1','CAA2','CAA3','CAA4','CAA5','CAA6','CAA7','CAA8','CAA9','CB1','CB2','CB3','CB4']
stations_2009  = ['L1','L1.1','L1.5','L2','L3','S4'] 
all_stations   = stations_2009 + stations_2015

dmn_ref_all    = np.empty((len(all_stations),50)); dmn_clean_all = np.empty((len(all_stations),50));
dmn_spm_all    = np.empty((len(all_stations),50)); dmn_bio_all   = np.empty((len(all_stations),50));
dmn_obs_all    = np.empty((len(all_stations),50));
dmn_ref_all[:] = np.nan; dmn_clean_all[:] = np.nan; dmn_spm_all[:] = np.nan; dmn_bio_all[:] = np.nan; dmn_obs_all[:] = np.nan;

for n,stn in enumerate(all_stations):
    dmn_ref, dmn_cleanice, dmn_spm, dmn_bio, dmn_ob = calc_scatter(stn)

    dmn_ref_all[n,0:len(dmn_ref)]        = dmn_ref
    dmn_clean_all[n,0:len(dmn_cleanice)] = dmn_cleanice
    dmn_spm_all[n,0:len(dmn_spm)]        = dmn_spm
    dmn_bio_all[n,0:len(dmn_bio)]        = dmn_bio
    dmn_obs_all[n,0:len(dmn_ob)]         = dmn_ob

In [ ]:
dmn_dirty   = dmn_ref_all
dmn_clean   = dmn_clean_all
dmn_rivers  = dmn_spm_all
dmn_biology = dmn_bio_all
dmn_observ  = dmn_obs_all

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(9.055, 3.74), dpi=300)
st = sns.axes_style("ticks")

with st:   
    for axis in ax:
        line1 = sns.lineplot([0, 20],[0, 20], c='k', linewidth=1.5, zorder=1, ax=axis)
        line1.lines[0].set_linestyle('--')
        axis.set_xlabel('Observed Mn [nM]', fontsize=8)
        axis.set_ylabel('Modelled Mn [nM]',fontsize=8)
        axis.set(xlim=(0, 14.), ylim=(0, 14.))
        axis.tick_params(axis='both', which='major', labelsize=8) 

    ### Left plot ###
    sns.scatterplot(dmn_observ.flatten(), dmn_dirty.flatten() , label=r'Reference', s=40, \
                    edgecolor='k', color=dirty, zorder=2, ax=ax[0])
    sns.scatterplot(dmn_observ.flatten(), dmn_clean.flatten() , label=r'Clean sea ice' , s=40, \
                     edgecolor='k', color=clean, zorder=2, ax=ax[0])
    sns.scatterplot([np.nanmean(dmn_observ)], [np.nanmean(dmn_dirty)], s=100, linewidth=1.0, \
                    edgecolor='k', marker='s', color=dirty, zorder=3, ax=ax[0])
    sns.scatterplot([np.nanmean(dmn_observ)], [np.nanmean(dmn_clean)], s=100, linewidth=1.0, \
                     edgecolor='k', marker='s', color=clean, zorder=3, ax=ax[0])
    ax[0].legend(loc=(0.01, 0.86), frameon=False, fontsize=8)
    
    ### Right plot ###
    sns.scatterplot(dmn_observ.flatten(), dmn_dirty.flatten() , label=r'Reference', s=40, \
                    edgecolor='k', color=dirty, zorder=2, ax=ax[1])
    sns.scatterplot(dmn_observ.flatten(), dmn_rivers.flatten(), label=r'Upper bound rivers', s=40, \
                     edgecolor='k', color=rivers, zorder=2, ax=ax[1])
    sns.scatterplot([np.nanmean(dmn_observ)], [np.nanmean(dmn_rivers)], s=100, linewidth=1.0, \
                     edgecolor='k', marker='s', color=rivers, zorder=3, ax=ax[1])
    sns.scatterplot([np.nanmean(dmn_observ)], [np.nanmean(dmn_dirty)], s=100, linewidth=1.0, \
                    edgecolor='k', marker='s', color=dirty, zorder=3, ax=ax[1])
    ax[1].legend(loc=(0.01, 0.86), frameon=False, fontsize=8)
    
#-----------------------------------------------------------------------------------------------------------------------------------
### Full figure ###
fig.text(0.08, 0.88, 'a)', fontsize=9);
fig.text(0.50, 0.88, 'b)', fontsize=9);

fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/R3-model-vs-observations.jpg', \
               bbox_inches='tight', dpi=300, transparent=True)
fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/R3-model-vs-observations.svg', \
               bbox_inches='tight', dpi=300, format='svg')